In [1]:
import psycopg2
import requests
import json
from tqdm import tqdm
import datetime
from datetime import date
from datetime import timedelta
import copy
import numpy as np
import math
import itertools

Psql - class to work with Postgresql

In [2]:
class Pgscl(object):
    def __init__(self):
        self.__conn = psycopg2.connect(dbname='inspirehep', user='postgres', password='qwerty', host='localhost')
        self.__cursor = self.__conn.cursor()
        self.__conn.autocommit = True
        print('Connection established')
    def __del__(self):
        self.__cursor.close()
        self.__conn.close()
        print('Connection closed')
    def insert(self, json):
        self.__cursor.execute("INSERT INTO articles_json VALUES(%s)", (json,))
    def clean(self):
        self.__cursor.execute("DELETE FROM articles_json")
        print('Deleted')
    def parse_json(self):
        self.__cursor.execute("CREATE TABLE IF NOT EXISTS articles (recid INTEGER, reference JSON, title TEXT, publication_date INT, authors JSON)")
        self.__cursor.execute("INSERT INTO articles SELECT (json->>'recid')::int, json->'reference', json->'title'->>'title', (json->'publication_info'->>'year')::int, json->'authors' FROM articles_json")
        return
    def del_dups(self):
        self.__cursor.execute("SELECT min(articles.ctid) as pgid, recid, count(articles.ctid) as num FROM articles " + 
                              "GROUP BY recid ORDER BY num DESC")
        res = copy.deepcopy(tuple(self.__cursor))
        for row in res:
            if row[2] == 1:
                return
            self.__cursor.execute("DELETE FROM articles WHERE (recid = %s) AND (articles.ctid != %s)", 
                                  (row[1], row[0]))
        return
    #поглощение (можно сделать чуть более общим)
    def Form_graph_absorb(self):
        self.__cursor.execute("SELECT recid FROM articles_nice")
        collab = np.zeros((7151, 7151))
        pbar = tqdm(total = 193172)
        res = copy.deepcopy(tuple(self.__cursor))
        for row in res:
            aff = []
            self.__cursor.execute("SELECT uni_ids FROM art_auth WHERE article_id = %s", (int(row[0]), ))
            for row1 in self.__cursor:
                aff += [row1[0]]
            for i in range(0, len(aff)):
                aff[i] = set(aff[i])
            for i in range(0, len(aff)):
                for j in range(0, len(aff)):
                    if aff[i] < aff[j]:
                        aff[j] = aff[i]
            i = 0
            while i < len(aff):
                for j in range(0, len(aff)):
                    if (i != j) and (aff[i] == aff[j]):
                        aff[j] = set()
                while aff.count(set()):
                    aff.remove(set())
                i = i + 1
            if (len(aff) > 1):
                tmp = np.zeros((7151, 7151))
                for i in range(0, len(aff)):
                    for j in range(i+1, len(aff)):
                        for lol in itertools.product(aff[i], aff[j]):
                            tmp[lol[0]][lol[1]] = 1
                collab += tmp
                del tmp
            pbar.update(1)
        for i in range(0, 7151):
            for j in range(i+1, 7151):
                if (collab[i][j] + collab[j][i] > 0):
                    self.__cursor.execute("INSERT INTO collab VALUES(%s, %s, %s)", (i, j, collab[i][j] + collab[j][i]))
        return
    #объединение
    def Form_graph_un(self):
        self.__cursor.execute("SELECT recid FROM articles_nice")
        collab = np.zeros((7151, 7151))
        pbar = tqdm(total = 193172)
        res = copy.deepcopy(tuple(self.__cursor))
        for row in res:
            aff = []
            self.__cursor.execute("SELECT uni_ids FROM art_auth WHERE article_id = %s", (int(row[0]), ))
            for row1 in self.__cursor:
                aff += [row1[0]]
            for i in range(0, len(aff)):
                aff[i] = set(aff[i])
            for i in range(0, len(aff)):
                for j in range(0, len(aff)):
                    if aff[i] & aff[j]:
                        aff[i] |= aff[j]
                        aff[j] = aff[i]
            i = 0
            while i < len(aff):
                for j in range(0, len(aff)):
                    if (i != j) and (aff[i] == aff[j]):
                        aff[j] = set()
                while aff.count(set()):
                    aff.remove(set())
                i = i + 1
            if (len(aff) > 1):
                tmp = np.zeros((7151, 7151))
                for i in range(0, len(aff)):
                    for j in range(i+1, len(aff)):
                        for lol in itertools.product(aff[i], aff[j]):
                            tmp[lol[0]][lol[1]] = 1
                collab += tmp
                del tmp
            pbar.update(1)
        for i in range(0, 7151):
            for j in range(i+1, 7151):
                if (collab[i][j] + collab[j][i] > 0):
                    self.__cursor.execute("INSERT INTO collab VALUES(%s, %s, %s)", (i+1, j+1, collab[i][j] + collab[j][i]))
        return
    #graph for two unis
    def Form_two_uni(self, uni1, uni2):
        u1 = None
        u2 = None
        self.__cursor.execute("SELECT aff_id FROM universities WHERE affiliation = %s", ("\"" + uni1 + "\"", ))
        for row in self.__cursor:
            u1 = row[0]
        if u1 is None:
            print("First institute does not exist.")
            return
        self.__cursor.execute("SELECT aff_id FROM universities WHERE affiliation = %s", ("\"" + uni2 + "\"", ))
        for row in self.__cursor:
            u2 = row[0]
        if u2 is None:
            print("Second institute does not exist.")
            return
        self.__cursor.execute("SELECT recid FROM articles_nice")
        res = copy.deepcopy(tuple(self.__cursor))
        pbar = tqdm(total = 193172)
        for row in res:
            auth = []
            self.__cursor.execute("SELECT author_id, uni_ids FROM art_auth WHERE article_id = %s", (int(row[0]), ))
            for row1 in self.__cursor:
                if row1[1].count(u1) != 0 or row1[1].count(u2) != 0:
                    auth.append(row1[0])
            if len(auth) > 1:
                for lol in itertools.product(auth, auth):
                    if lol[0] < lol[1]:
                        self.__cursor.execute("INSERT INTO collab1 VALUES(%s, %s, 1) " + \
                                              "ON CONFLICT ON CONSTRAINT collab1_pkey DO " + \
                                              "UPDATE SET num = collab1.num + excluded.num", 
                                              (lol[0], lol[1]))
            pbar.update(1)
        return
        

Code that forms graph table (source, target, weight)

In [ ]:
try:
    pg = Pgscl()
    pg.Form_graph_absorb()
finally:
    del pg

In [3]:
try:
    pg = Pgscl()
    pg.Form_two_uni("CERN", "Fermilab")
finally:
    del pg

Connection established


100%|████████████████████████████████████████████████████████████████████████| 193172/193172 [6:12:01<00:00,  8.91it/s]

Connection closed


Code that downloads to Postgresql articles written in 2012-2019 years

In [ ]:
pg = Pgscl()
try:
    for year in range(2012,2020):
        i = 0
        while 1:
            url = 'https://inspirehep.net/search?p=find jy ' + str(year) + \
'&of=recjson&ot=recid,primary_report_number,title,creation_date,publication_info,reference,authors&rg=250&jrec=' + \
                str(i+1)
            response = requests.get(url)
            if response.content != b'':
                kuku = response.content.decode('UTF-8')
                raw = response.json() 
                for raw_one in raw:
                    i += 1
                    pg.insert(json.dumps(raw_one))
                if i % 250:
                    break
    pg.del_dups()
finally:
    del pg

Code that deletes duplicates (faster than in postgres)

In [ ]:
pg = Pgscl()
try:
    pg.del_dups()
finally:
    del pg